<a href="https://colab.research.google.com/github/GeorgeTelles/RSI-Backtesting/blob/main/Backtesting_RSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Importando Bibliotecas e Modulos

In [ ]:
!pip install vectorbt
!pip install ta

In [ ]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import vectorbt as vbt
import ta
import warnings
warnings.filterwarnings('ignore')

# 2. Obtendo dados dos Ativos

In [ ]:
ativo = "ITUB4.SA"

dados_ohlc = yf.download(ativo, "2023-1-1")

#3. Tratando, modelando e separando os dados

In [ ]:
proporcao_treino = 0.70

# Calcular o índice de separação
total_linhas = len(dados_ohlc)
indice_separacao = int(total_linhas * proporcao_treino)

# Dividir o DataFrame
dados_teste = dados_ohlc.iloc[:indice_separacao]
dados_valid = dados_ohlc.iloc[indice_separacao:]

# Verificar os tamanhos dos conjuntos
print(f"Tamanho do conjunto de teste: {len(dados_teste)}")
print(f"Tamanho do conjunto de validação: {len(dados_valid)}")

#4. Definindo metricas a serem testadas

In [ ]:
periodo_rsi = list(range(7, 22, 1))
niveis_entrada = list(range(10, 51, 5))
niveis_saida = list(range(50, 91, 5))


#5. Loop de testes com dados de Teste

In [ ]:
lista_resultados = []
lista_backtest = []

for periodo in periodo_rsi:
  dados_teste2 = dados_teste.copy()
  rsi = ta.momentum.RSIIndicator(dados_teste2['Close'], window = periodo, fillna = False)
  dados_teste2['RSI'] = rsi.rsi()
  dados_teste2 = dados_teste2.dropna()
  for i in niveis_entrada:
        for j in niveis_saida:

          entradas = dados_teste2['RSI'] < i

          saidas = dados_teste2['RSI'] > j

          backtest = vbt.Portfolio.from_signals(dados_teste2['Close'],
                                                    entradas,
                                                    saidas,
                                                    direction='longonly',
                                                    size_type='Amount', size=1)
          if backtest.stats()['Total Return [%]'] > 0:
            lista_resultados.append([periodo, i, j, backtest.stats()['Total Return [%]'], backtest.stats()['Benchmark Return [%]'] ])
            lista_backtest.append(backtest)


# 6. Conferindo os 10 melhores resultados

In [ ]:
resultados = pd.DataFrame(lista_resultados, columns=['Periodo','Entrada', 'Saida', 'Resultado', 'Resultado Holding'])
top10 = resultados.sort_values(by='Resultado', ascending=False).head(10)
top10

#7. Visualização individual do teste

In [ ]:
lista_backtest[312].plot().show()

In [ ]:
lista_backtest[312].stats()

#8. Teste Apenas com os parametros top10 dos dados de teste

In [ ]:
periodos_top = top10['Periodo'].tolist()
entradas_top = top10['Entrada'].tolist()
saidas_top = top10['Saida'].tolist()

In [ ]:
lista_resultados_teste = []
lista_backtest_teste = []

for i in range(len(entradas_top)):

  dados_valid2 = dados_valid.copy()
  rsi = ta.momentum.RSIIndicator(dados_valid2['Close'], window = periodos_top[i], fillna = False)
  dados_valid2['RSI'] = rsi.rsi()
  dados_valid2 = dados_valid2.dropna()

  entradas = dados_valid2['RSI'] < entradas_top[i]

  saidas = dados_valid2['RSI'] > saidas_top[i]

  backtest = vbt.Portfolio.from_signals(dados_valid2['Close'],
                                            entradas,
                                            saidas,
                                            direction='longonly',
                                            size_type='Amount', size=1)
  lista_resultados_teste.append([periodos_top[i], entradas_top[i], saidas_top[i], backtest.stats()['Total Return [%]'], backtest.stats()['Benchmark Return [%]'] ])
  lista_backtest.append(backtest)

In [ ]:
resultados_teste = pd.DataFrame(lista_resultados_teste, columns=['Periodo', 'Entrada', 'Saida', 'Resultado', 'Resultado Holding'])
top10_teste = resultados_teste.sort_values(by='Resultado', ascending=False).head(10)
top10_teste

#9. Backtesting com dados de Validação

In [ ]:
lista_resultados_valid = []
lista_backtest_valid = []

for periodo in periodo_rsi:
  dados_valid2 = dados_valid.copy()
  rsi = ta.momentum.RSIIndicator(dados_valid2['Close'], window = periodo, fillna = False)
  dados_valid2['RSI'] = rsi.rsi()
  dados_valid2 = dados_valid2.dropna()
  for i in niveis_entrada:
        for j in niveis_saida:

          entradas = dados_valid2['RSI'] < i

          saidas = dados_valid2['RSI'] > j

          backtest = vbt.Portfolio.from_signals(dados_valid2['Close'],
                                                    entradas,
                                                    saidas,
                                                    direction='longonly',
                                                    size_type='Amount', size=1)
          if backtest.stats()['Total Return [%]'] > 0:
            lista_resultados_valid.append([periodo, i, j, backtest.stats()['Total Return [%]'], backtest.stats()['Benchmark Return [%]'] ])
            lista_backtest_valid.append(backtest)


In [ ]:
resultados_valid = pd.DataFrame(lista_resultados_valid, columns=['Periodo','Entrada', 'Saida', 'Resultado', 'Resultado Holding'])
top10_valid = resultados_valid.sort_values(by='Resultado', ascending=False).head(10)
top10_valid

**Verificando se existem parametros iguais entre os top 10 dos dados de teste e o top 10 dos dados de validação**

In [ ]:
common_rows = pd.merge(top10_valid, top10, on=['Periodo', 'Entrada', 'Saida'])

# Imprimir as linhas comuns
print("Linhas que são iguais em ambos os dataframes:")
common_rows